# About
- Get list of youtube videos in the nutrition facts channel
- Download the transcripts of the videos

In [1]:
import os
import pandas as pd

from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
# from youtube_transcript_api._errors import TranscriptsDisabled
from dotenv import load_dotenv

# Get api keys
load_dotenv()
YOUTUBE_APIKEY = os.environ['YOUTUBE_APIKEY']

In [2]:
# Initialiaze youtube api client
youtube = build('youtube', 'v3', developerKey=YOUTUBE_APIKEY)

# Get Channel ID and List of Videos + Metadata

In [3]:
# Let's see what's available in the channels list api
#https://developers.google.com/youtube/v3/docs/channels/list
username = 'NutritionFactsOrg'
request = youtube.channels().list(
        part="id",
        forUsername=username
    )
response = request.execute()
response

{'kind': 'youtube#channelListResponse',
 'etag': 'y2wbGNYxO1mQLSW8NgLvKwMoFvQ',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'QrBFR4pa4iZJUXj0kXL8bnskofE',
   'id': 'UCddn8dUxYdgJz3Qr5mjADtA'}]}

In [4]:
# Let's see what's available in the channels list api
username = 'NutritionFactsOrg'
request = youtube.channels().list(
        part="statistics",
        forUsername=username
    )
response = request.execute()
response

{'kind': 'youtube#channelListResponse',
 'etag': 'v8DCiZDKPGEgSEXfwZU5e-r_cHo',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '9V35LcYYDQas4EuOmZyBgYqZs-M',
   'id': 'UCddn8dUxYdgJz3Qr5mjADtA',
   'statistics': {'viewCount': '250015034',
    'subscriberCount': '1160000',
    'hiddenSubscriberCount': False,
    'videoCount': '2678'}}]}

In [5]:
# Get the channel ID
channel_id = response['items'][0]['id']
channel_id

'UCddn8dUxYdgJz3Qr5mjADtA'

In [6]:
# Get a list of videos in the given channel
response = youtube.channels().list(part='contentDetails', id=channel_id).execute()
playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

# Get a list of videos in the channel's uploads playlist
videos = []
next_page_token = None
while True:
    if next_page_token is None:
        response = youtube.playlistItems().list(part='snippet', playlistId=playlist_id, maxResults=50).execute()
    else:
        response = youtube.playlistItems().list(part='snippet', playlistId=playlist_id, maxResults=50, pageToken=next_page_token).execute()

    for item in response['items']:
        videos.append({
            'videoId': item['snippet']['resourceId']['videoId'],
            'title': item['snippet']['title'],
            'description': item['snippet']['description'],
            'publishedAt': item['snippet']['publishedAt']
        })

    next_page_token = response.get('nextPageToken')
    if next_page_token is None:
        break

# Create a DataFrame from the list of videos
df = pd.DataFrame(videos)

df

,videoId,title,description,publishedAt
0,7q3U_x_a6cA,Is FDA-Approved Bioidentical Hormone Replaceme...,"As of 2019, there are urine-free FDA-approved ...",2024-08-19T11:59:57Z
1,2zqc8vrMRqM,Friday Favorite: Benefits of Garlic for Fighti...,"Raw garlic is compared to roasted, stir-fried,...",2024-08-16T11:59:56Z
2,CLfiJNWuZ84,Podcast: Fluoridation in Water (Part 2),Why I changed my mind on fluoridation. This ep...,2024-08-15T11:59:53Z
3,x9d4KwxNwi4,I recommend eating nuts at least 3 times/week ...,New subscribers to our e-newsletter always rec...,2024-08-15T05:00:37Z
4,lV261tApWVg,How to Prevent Wrinkles with Diet,The evidence supports the recommendation to fo...,2024-08-14T11:59:54Z
...,...,...,...,...
2674,P_X3exQtuGA,The Healthiest Herbal Tea,New subscribers to our e-newsletter always rec...,2011-02-08T14:59:42Z
2675,ce-pvksbiwM,Update on Yerba Maté,New subscribers to our e-newsletter always rec...,2011-02-08T14:57:18Z
2676,1Yb5MjU38ng,Update on MSG,New subscribers to our e-newsletter always rec...,2011-02-08T14:48:31Z
2677,lD2RzNJYGxQ,Update on Juice Plus+®,New subscribers to our e-newsletter always rec...,2011-02-08T05:41:12Z


# Get the transcripts

In [7]:
def get_transcript(video_id):
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=['en', "English"])
        # Combine the text of all segments into one large string
        transcript_text = ' '.join([segment['text'].replace('\n',' ') for segment in transcript_list])
        return transcript_text
    except Exception as e:
        print(f"Error for video {video_id}: {e}")
        return None

In [9]:
# Get all the transcripts for 2020 - present
df['transcript'] = df['videoId'].apply(get_transcript)
# Save the transcripts
df.to_csv('../data/transcripts_all_2024-08-19')

Error for video wLADCxY1_r8: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=wLADCxY1_r8! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Error for video P1IGY-UYETE: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=P1IGY-UYETE! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Pl